# KIWIBOT 
## IMPOVING THE TRAVEL INDUSTRY ONE CHAT AT A TIME!

![alt text](https://steemitimages.com/640x0/https://cdn.steemitimages.com/DQmZCZr4ewvhDsQJy63yKC8D8D8cLrKae1L1hEWDsTxPBaG/3q-500x500.png)

This notebook is creates a chatbot for flight information website [KIWI](https://www.kiwi.com/en/)

The notebook consists of three parts:
*   Part 0: Installation and preparations
*   Part 1: Needed files to create the bot
*   Part 1: Start with a basic bot that can only understand natural language but no dialogues.
*   Part 2: Add the abilitiy to understand multiturn dialogues.
*   Part 3: Further resources so you can extend this simple demo.

## Part 0: Installation

### Let's start with jupyter configuration

In [1]:
!pip install matplotlib
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

### Installation of Rasa
First you'll have to install Rasa in this notebook if you already have it installed in your env, you can just skip this. As dependencies, it will install Tensorflow and sklearn-crfsuite among others.

In [2]:
#!pip install rasa

### Part 1 Needed files
The most important files are marked with a *.

<style>
td {
  font-size: 50px
}
</style>

| Filename  | Description |
| --- | --- |
| actions.py  | code for your custom actions  |
| config.yml*  | configuration of your NLU and Core models  |
| credentials.yml | details for connecting to other services
| data/nlu.md* | your NLU training data
| data/stories.md* | your stories |
| domain.yml* | your assistant’s domain |
| endpoints.yml	| details for connecting to channels like fb messenger |


In [3]:
!mkdir data/
nlu_md = """

## intent:flight
- hey
- hello
- flight details
- hey! I want latest flight schedule
- can you provide me flight schedule
- please provide latest flight info
- is it possible to get latest flight information
- flight schedule
- flight information
- flight info
- hi 
- flight help 
- booking my flight 


## intent:inform
- [MUC](location)
- [CPT](location)
- [BCN](location)
- [BRU](location)
- [BER](location)
- [ATH](location)
- [ROM](location)
- [MIL](location)
- [LIS](location)
- [WAW](location)
- [MAD](location)
- [PAR](location)
- [LON](location)

- [09/06/2019](date)
- I want for [09/06/2019](date)
- I want to depart on the [09/06/2019](date)
- I want the leave on the [09/06/2019](date)
- I want to travel on [21/06/2019](date)
- I would like to leave on [21/06/2019](date)

## intent:affirmation
- ok
- perfect
- lovely
- great
- yea
- definitely
- yah
- yep
- ya
- sure
- yes
- yes please 
- YES

## intent:deny
- no
- not yet
- never
- not now
- negative 
- nope
- no thank you
- NO

"""

%store nlu_md > data/nlu.md

Writing 'nlu_md' (str) to file 'data/nlu.md'.


In [4]:
config ="""
# Configuration for Rasa NLU.
# https://rasa.com/docs/rasa/nlu/components/
language: en
pipeline: supervised_embeddings

# Configuration for Rasa Core.
# https://rasa.com/docs/rasa/core/policies/
policies:
  - name: MemoizationPolicy
  - name: KerasPolicy
    epochs: 200
  - name: MappingPolicy
"""
%store config > config.yml

Writing 'config' (str) to file 'config.yml'.


In [5]:
!rasa train nlu

Training NLU model...
2019-06-12 15:16:45 INFO     rasa.nlu.training_data.loading  - Training data format of /var/folders/l3/qm78fw951vg5gc3cd04wm8m80000gn/T/tmpv_soukf2/31a9e18cb0b348228626b30d3b32b8c2_nlu.md is md
2019-06-12 15:16:45 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 53 (4 distinct intents)
	- Found intents: 'deny', 'flight', 'inform', 'affirmation'
	- entity examples: 19 (2 distinct entities)
	- found entities: 'location', 'date'

2019-06-12 15:16:45 INFO     rasa.nlu.model  - Starting to train component WhitespaceTokenizer
2019-06-12 15:16:45 INFO     rasa.nlu.model  - Finished training component.
2019-06-12 15:16:45 INFO     rasa.nlu.model  - Starting to train component RegexFeaturizer
2019-06-12 15:16:45 INFO     rasa.nlu.model  - Finished training component.
2019-06-12 15:16:45 INFO     rasa.nlu.model  - Starting to train component CRFEntityExtractor
2019-06-12 15:16:45 INFO     rasa.nlu.model  - Finished training componen

In [15]:
#rasa run --enable-api -m models/nlu-20190612-151649.tar.gz
!curl localhost:5005/model/parse -d '{"text":"flight"}'

{"intent":{"name":"flight","confidence":0.9602780342},"entities":[],"intent_ranking":[{"name":"flight","confidence":0.9602780342},{"name":"affirmation","confidence":0.0},{"name":"inform","confidence":0.0},{"name":"deny","confidence":0.0}],"text":"flight"}

In [16]:
#!mkdir models/nlu
#!tar -zxf models/nlu-20190611-184922.tar.gz --directory models/nlu
#from rasa.nlu.model import Interpreter

#interpreter = Interpreter.load('models/nlu/nlu')
# Parse message to get result.
#print(interpreter.parse("doing great"))

In [17]:
!rasa test nlu

2019-06-12 15:20:50 INFO     rasa.nlu.training_data.loading  - Training data format of /var/folders/l3/qm78fw951vg5gc3cd04wm8m80000gn/T/tmp1m56zjty/30e0c65e2a854f7d86fc1ed7ee5496f8_nlu.md is md
2019-06-12 15:20:50 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 53 (4 distinct intents)
	- Found intents: 'affirmation', 'deny', 'flight', 'inform'
	- entity examples: 19 (2 distinct entities)
	- found entities: 'date', 'location'

2019-06-12 15:20:50 INFO     rasa.nlu.test  - Running model for predictions:
100%|██████████████████████████████████████████| 53/53 [00:00<00:00, 709.01it/s]
2019-06-12 15:20:50 INFO     rasa.nlu.test  - Intent evaluation results:
2019-06-12 15:20:50 INFO     rasa.nlu.test  - Intent Evaluation: Only considering those 53 examples that have a defined intent out of 53 examples
2019-06-12 15:20:50 INFO     rasa.nlu.test  - F1-Score:  1.0
2019-06-12 15:20:50 INFO     rasa.nlu.test  - Precision: 1.0
2019-06-12 15:20:50 INFO    

# Part 2: Adding dialogue capabilities
### Writing Stories

In [18]:
stories_md = """
## fallback
- utter_unclear

## Story 1
* flight
    - utter_boarding
* inform{"location": "BCN"}
    - action_save_origin
    - slot{"from": "BCN"}
    - utter_destination
* inform{"location": "MUC"}
    - action_save_destination
    - slot{"to": "MUC"}
    - utter_date
* inform{"date": "20/06/2019"}
    - action_save_date
    - slot{"date": "20/06/2019"}
    - utter_confirm
* affirmation
    - action_get_flight
    - utter_check_another_one
* deny
    - utter_thanks
    - action_restart

## Stry 2-multiple steps
* flight
    - utter_boarding
* inform{"location": "CPT"}
    - action_save_origin
    - slot{"from": "CPT"}
    - utter_destination
* inform{"location": "BCN"}
    - action_save_destination
    - slot{"to": "BCN"}
    - utter_date
* inform{"date": "03/07/2019"}
    - slot{"date": "03/07/2019"}
    - action_save_date
    - slot{"date": "03/07/2019"}
    - utter_confirm
* affirmation
    - action_get_flight
    - utter_check_another_one
* affirmation
    - action_slot_reset
    - reset_slots
    - utter_boarding
* inform{"location": "BCN"}
    - action_save_origin
    - slot{"from": "BCN"}
    - utter_destination
* inform{"location": "MUC"}
    - action_save_destination
    - slot{"to": "MUC"}
    - utter_date
* inform{"date": "10/07/2019"}
    - slot{"date": "10/07/2019"}
    - action_save_date
    - slot{"date": "10/07/2019"}
    - utter_confirm
* affirmation
    - action_get_flight
    - utter_check_another_one
* deny
    - utter_thanks
    - action_restart

"""

%store stories_md > data/stories.md

Writing 'stories_md' (str) to file 'data/stories.md'.


### Defining a Domain

The domain specifies the universe that your bot lives in. You should list all of the intents and actions that show up in your stories. 
This is also the place to write templates, which contain the messages your bot can send back

In [19]:
domain_yml = """

actions:
- utter_boarding
- utter_destination
- utter_date
- utter_confirm
- utter_check_another_one
- utter_thanks
- utter_unclear
- action_save_origin
- action_save_destination
- action_save_date
- action_get_flight
- action_slot_reset

config:
  store_entities_as_slots: true
entities:
- location
- date
intents:
- ticket:
    use_entities: true
- inform:
    use_entities: true
- affirmation:
    use_entities: true
- deny:
    use_entities: true

slots:
  date:
    initial_value: null
    type: rasa.core.slots.TextSlot
  from:
    initial_value: null
    type: rasa.core.slots.TextSlot
  to:
    initial_value: null
    type: rasa.core.slots.TextSlot

templates:
  utter_boarding:
  - text: We'll help you find the latest flight schedule. First, please provide your
      origin airport code?
  utter_check_another_one:
  - text: Do you want to make another inquiry?
  utter_confirm:
  - text: I will be making inquiry for flight from {from} to {to} on {date}. Is that
      correct?
  utter_date:
  - text: What is the date for your travel(in dd/mm/yyyy)?
  utter_destination:
  - text: And the destination airport code?
  utter_thanks:
  - text: Thanks for contacting us. Have a good day!
  utter_unclear:
  - text: I am not sure what you are aiming for.Kindly try it again
"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


### Adding Custom API methods

In [20]:
actions= """

from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

from rasa.core.domain import Domain
from rasa.core.trackers import EventVerbosity

import logging
logger = logging.getLogger(__name__)

import requests
import json

from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet
from rasa_core_sdk.events import UserUtteranceReverted
from rasa_core_sdk.events import AllSlotsReset
from rasa_core_sdk.events import Restarted

from bs4 import BeautifulSoup
import urllib.request
import re
import requests

class SaveOrigin(Action):
    def name(self):
        return 'action_save_origin'

    def run(self, dispatcher, tracker, domain):
        orig = next(tracker.get_latest_entity_values("location"), None)
        if not orig:
            dispatcher.utter_message("Please enter a valid airport code")
            return [UserUtteranceReverted()]
        return [SlotSet('from',orig)]
    
class SaveDestination(Action):
    def name(self):
        return 'action_save_destination'

    def run(self, dispatcher, tracker, domain):
        dest = next(tracker.get_latest_entity_values("location"), None)
        if not dest:
            dispatcher.utter_message("Please enter a valid airport code")
            return [UserUtteranceReverted()]
        return [SlotSet('to',dest)]

class SaveDate(Action):
    def name(self):
        return 'action_save_date'

    def run(self, dispatcher, tracker, domain):
        inp = next(tracker.get_latest_entity_values("date"), None)
        if not inp:
            dispatcher.utter_message("Please enter a valid date")
            return [UserUtteranceReverted()]
        return [SlotSet('date',inp)]

class ActionSlotReset(Action): 	
    def name(self): 
        return 'action_slot_reset' 	
    def run(self, dispatcher, tracker, domain): 
        return[AllSlotsReset()]

class getFlightStatus(Action):
    def name(self):
        return 'action_get_flight'
    def run(self, dispatcher, tracker, domain):
        orig=tracker.get_slot('from')
        dest=tracker.get_slot('to')
        dat=tracker.get_slot('date')
        dispatcher.utter_message("Here is the link to your flight booking")
        urls = ("https://api.skypicker.com/flights?"+
                       "flyFrom=" + orig +
                       "&to="+ dest +
                       "&dateFrom=" + dat + 
                       "&partner=picky") 
        response = requests.get(urls)
        data = response.text
        parsed = json.loads(data) 
        class Test(object):
            def __init__(self, data):
                self.__dict__ = json.loads(data)
        flight_data = Test(data)
        flight_data = flight_data.data[1]['deep_link']
        dispatcher.utter_message(flight_data)
        return []
"""

%store actions > actions.py

endpoints = """
action_endpoint:
  url: http://localhost:5055/webhook
"""
%store endpoints > endpoints.yml

Writing 'actions' (str) to file 'actions.py'.
Writing 'endpoints' (str) to file 'endpoints.yml'.


## Training your Dialogue Model

In [21]:
!rasa train

Nothing changed. You can use the old model stored at '/Users/17867029/Desktop/kiwibot/models/20190612-151848.tar.gz'.


### Pro Tip: Visualising the Training Data

You can visualise the stories to get a sense of how the conversations go. This is usually a good way to see if there are any stories which don't make sense


In [22]:
#you may need any of these to be able to display the conversations graph
#!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
#!brew install graphviz;
!rasa visualize

2019-06-12 15:21:00 INFO     rasa.nlu.training_data.loading  - Training data format of /var/folders/l3/qm78fw951vg5gc3cd04wm8m80000gn/T/tmp2lhjyzne/f18f1048bb814bfdb6782ff3bd4a815c_nlu.md is md
2019-06-12 15:21:00 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 53 (4 distinct intents)
	- Found intents: 'affirmation', 'flight', 'deny', 'inform'
	- entity examples: 19 (2 distinct entities)
	- found entities: 'location', 'date'

2019-06-12 15:21:00 INFO     rasa.core.visualize  - Starting to visualize stories...
2019-06-12 15:21:00 WARNING  rasa.core.training.dsl  - Found unknown intent 'flight' on line 6. Please, make sure that all intents are listed in your domain yaml.
2019-06-12 15:21:00 WARNING  rasa.core.training.dsl  - Found unknown intent 'flight' on line 28. Please, make sure that all intents are listed in your domain yaml.
Processed Story Blocks: 100%|█████| 3/3 [00:00<00:00, 1696.95it/s, # trackers=1]
2019-06-12 15:21:00 INFO     rasa.

## Testing the bot
First run the actions endpoint server and then run "rasa shell" in the terminal.


In [23]:
%%script bash --bg 
rasa run actions

In [268]:
#this must run in the reminal
#!rasa shell

In [269]:
#When its finished, we can stop all background scripts with
%killbgscripts

All background processes were killed.


# END